# Imports et Configuration
Import des bibliothèques nécessaires (transformers, torch) et configuration initiale des modèles.

In [8]:
# Imports et Configuration

# Import des bibliothèques nécessaires
import torch
from transformers import pipeline, CamembertTokenizer, CamembertForTokenClassification

# Configuration initiale des modèles
# Modèle pour l'analyse grammaticale
pos_model_name = "qanastek/pos-french-camembert"
pos_tokenizer = CamembertTokenizer.from_pretrained(pos_model_name)
pos_model = CamembertForTokenClassification.from_pretrained(pos_model_name)
pos_pipeline = pipeline("ner", model=pos_model, tokenizer=pos_tokenizer)

# Modèle pour choisir le mot à insérer
fill_mask_model_name = "camembert-base"
fill_mask_pipeline = pipeline("fill-mask", model=fill_mask_model_name)

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing CamembertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Chargement des Modèles Transformers
Chargement du modèle qanastek/pos-french pour l'analyse grammaticale et d'un modèle de langue (camembert-base) pour la sélection des mots.

In [9]:
# Chargement des Modèles Transformers

# Chargement du modèle qanastek/pos-french pour l'analyse grammaticale
pos_model_name = "qanastek/pos-french-camembert"
pos_tokenizer = CamembertTokenizer.from_pretrained(pos_model_name)
pos_model = CamembertForTokenClassification.from_pretrained(pos_model_name)
pos_pipeline = pipeline("ner", model=pos_model, tokenizer=pos_tokenizer)

# Chargement du modèle camembert-base pour la sélection des mots
fill_mask_model_name = "camembert-base"
fill_mask_pipeline = pipeline("fill-mask", model=fill_mask_model_name)

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing CamembertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Fonction de Détection des Verbes
Implémentation de la fonction qui utilise le modèle POS pour identifier les verbes dans les proverbes modifiés.

In [10]:
def detect_verbs(proverb):
    """
    Utilise le modèle POS pour identifier les verbes dans un proverbe modifié.
    
    Args:
    proverb (str): Le proverbe modifié.
    
    Returns:
    List[str]: Liste des verbes détectés dans le proverbe.
    """
    # Utilisation du pipeline POS pour obtenir les étiquettes grammaticales
    pos_tags = pos_pipeline(proverb)
    
    # Extraction des verbes à partir des étiquettes POS
    verbs = [word['word'] for word in pos_tags if word['entity'] == 'VERB']
    
    return verbs

# Exemple d'utilisation de la fonction
proverb_example = "a beau mentir qui part de loin"
detected_verbs = detect_verbs(proverb_example)
print(detected_verbs)

['▁a', '▁mentir', '▁part']


# Fonction de Sélection du Meilleur Mot
Développement de la fonction qui utilise le modèle de langue pour choisir le meilleur mot parmi les candidats en fonction du contexte.

In [11]:
def select_best_word(proverb, word_list):
    """
    Utilise le modèle de langue pour choisir le meilleur mot parmi les candidats en fonction du contexte du proverbe.
    
    Args:
    proverb (str): Le proverbe modifié.
    word_list (List[str]): Liste des mots candidats.
    
    Returns:
    str: Le meilleur mot choisi en fonction du contexte.
    """
    # Génération des prédictions pour chaque mot candidat
    predictions = []
    for word in word_list:
        masked_proverb = proverb.replace("[MASK]", "[MASK]")
        prediction = fill_mask_pipeline(masked_proverb)
        for pred in prediction:
            if pred['token_str'] == word:
                predictions.append((word, pred['score']))
                break
    
    # Sélection du mot avec le score le plus élevé
    best_word = max(predictions, key=lambda x: x[1])[0]
    
    return best_word

# Exemple d'utilisation de la fonction
proverb_example = "a beau [MASK] qui part de loin"
word_candidates = ["vient", "revient"]
best_word = select_best_word(proverb_example, word_candidates)
print(best_word)

PipelineException: No mask_token (<mask>) found on the input

# Évaluation et Analyse des Résultats
Évaluation des performances du système sur le jeu de test et analyse détaillée des erreurs et des succès.

In [ ]:
# Fonction pour évaluer les performances du système sur le jeu de test
def evaluate_system(test_data):
    correct_predictions = 0
    total_predictions = len(test_data)
    
    for item in test_data:
        masked_proverb = item['Masked']
        word_list = item['Word_list']
        correct_proverb = item['Proverb']
        
        # Détection des verbes dans le proverbe masqué
        verbs = detect_verbs(masked_proverb)
        
        # Si des verbes sont détectés, remplacer le premier verbe détecté par [MASK]
        if verbs:
            masked_proverb = masked_proverb.replace(verbs[0], "[MASK]")
        
        # Sélection du meilleur mot parmi les candidats
        best_word = select_best_word(masked_proverb, word_list)
        
        # Reconstruction du proverbe avec le meilleur mot sélectionné
        corrected_proverb = masked_proverb.replace("[MASK]", best_word)
        
        # Vérification si le proverbe corrigé est correct
        if corrected_proverb == correct_proverb:
            correct_predictions += 1
    
    accuracy = correct_predictions / total_predictions
    return accuracy

# Évaluation du système sur le jeu de test
accuracy = evaluate_system(tests)
print(f"Accuracy: {accuracy:.2f}")

# Analyse des résultats
def analyze_results(test_data):
    for item in test_data:
        masked_proverb = item['Masked']
        word_list = item['Word_list']
        correct_proverb = item['Proverb']
        
        # Détection des verbes dans le proverbe masqué
        verbs = detect_verbs(masked_proverb)
        
        # Si des verbes sont détectés, remplacer le premier verbe détecté par [MASK]
        if verbs:
            masked_proverb = masked_proverb.replace(verbs[0], "[MASK]")
        
        # Sélection du meilleur mot parmi les candidats
        best_word = select_best_word(masked_proverb, word_list)
        
        # Reconstruction du proverbe avec le meilleur mot sélectionné
        corrected_proverb = masked_proverb.replace("[MASK]", best_word)
        
        # Affichage des résultats
        print(f"Masked Proverb: {masked_proverb}")
        print(f"Word List: {word_list}")
        print(f"Correct Proverb: {correct_proverb}")
        print(f"Corrected Proverb: {corrected_proverb}")
        print(f"Is Correct: {corrected_proverb == correct_proverb}")
        print("-" * 50)

# Analyse détaillée des résultats
analyze_results(tests)